In [8]:
import pandas as pd
import numpy as np

from flask import Flask, jsonify, render_template
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from pandas_geojson import to_geojson

In [9]:
engine = create_engine("sqlite:///sqlite3/melbournebusinessdatabase.db")
database_df = pd.read_sql("Select * from melbourne_business where census_year = 2010",engine)
# database_df = database_df.iloc[: , 1:]
database_df['number_of_seats'] = database_df['number_of_seats'].astype('int')
database_df


,Unnamed: 0,census_year,clue_small_area,trading_name,industry_anzsic4_description,seating_type,number_of_seats,x_coordinate,y_coordinate
0,17767,2010,Melbourne (CBD),26 Spencer Street MELBOURNE 3000,Cafes and Restaurants,Seats - Outdoor,36,144.9575,-37.8211
1,17768,2010,Melbourne (CBD),26 Spencer Street MELBOURNE 3000,Cafes and Restaurants,Seats - Indoor,32,144.9575,-37.8211
2,17769,2010,Melbourne (CBD),20 Spencer Street MELBOURNE 3000,Takeaway Food Services,Seats - Indoor,10,144.9575,-37.8211
3,17770,2010,Melbourne (CBD),14 Spencer Street MELBOURNE 3000,Takeaway Food Services,Seats - Outdoor,4,144.9575,-37.8211
4,17771,2010,Melbourne (CBD),557-557 Flinders Street MELBOURNE 3000,Takeaway Food Services,Seats - Outdoor,4,144.9565,-37.821
...,...,...,...,...,...,...,...,...,...
2828,20598,2010,Kensington,188 Bellair Street KENSINGTON 3031,Cafes and Restaurants,Seats - Indoor,41,144.93,-37.7939
2829,20599,2010,Kensington,174 Bellair Street KENSINGTON 3031,Cafes and Restaurants,Seats - Outdoor,12,144.9299,-37.7936
2830,20600,2010,Kensington,174 Bellair Street KENSINGTON 3031,Cafes and Restaurants,Seats - Indoor,14,144.9299,-37.7936
2831,20601,2010,Kensington,170 Bellair Street KENSINGTON 3031,Cafes and Restaurants,Seats - Outdoor,17,144.9299,-37.7935


In [10]:
not_top7 = database_df.groupby('clue_small_area').sum().sort_values('number_of_seats',ascending = False).index[7:]
not_top7

Index(['Kensington', 'Parkville', 'West Melbourne (Residential)',
       'Port Melbourne', 'South Yarra', 'West Melbourne (Industrial)'],
      dtype='object', name='clue_small_area')

In [11]:
database_df = database_df.replace(not_top7, 'Other')

In [12]:
database_df


,Unnamed: 0,census_year,clue_small_area,trading_name,industry_anzsic4_description,seating_type,number_of_seats,x_coordinate,y_coordinate
0,17767,2010,Melbourne (CBD),26 Spencer Street MELBOURNE 3000,Cafes and Restaurants,Seats - Outdoor,36,144.9575,-37.8211
1,17768,2010,Melbourne (CBD),26 Spencer Street MELBOURNE 3000,Cafes and Restaurants,Seats - Indoor,32,144.9575,-37.8211
2,17769,2010,Melbourne (CBD),20 Spencer Street MELBOURNE 3000,Takeaway Food Services,Seats - Indoor,10,144.9575,-37.8211
3,17770,2010,Melbourne (CBD),14 Spencer Street MELBOURNE 3000,Takeaway Food Services,Seats - Outdoor,4,144.9575,-37.8211
4,17771,2010,Melbourne (CBD),557-557 Flinders Street MELBOURNE 3000,Takeaway Food Services,Seats - Outdoor,4,144.9565,-37.821
...,...,...,...,...,...,...,...,...,...
2828,20598,2010,Other,188 Bellair Street KENSINGTON 3031,Cafes and Restaurants,Seats - Indoor,41,144.93,-37.7939
2829,20599,2010,Other,174 Bellair Street KENSINGTON 3031,Cafes and Restaurants,Seats - Outdoor,12,144.9299,-37.7936
2830,20600,2010,Other,174 Bellair Street KENSINGTON 3031,Cafes and Restaurants,Seats - Indoor,14,144.9299,-37.7936
2831,20601,2010,Other,170 Bellair Street KENSINGTON 3031,Cafes and Restaurants,Seats - Outdoor,17,144.9299,-37.7935


In [13]:
gpA_df = database_df.groupby('clue_small_area', as_index=False).sum().sort_values('number_of_seats')
gpA_df

,clue_small_area,number_of_seats
5,North Melbourne,4976
2,East Melbourne,6515
4,Melbourne (Remainder),8792
7,Southbank,11004
6,Other,12693
0,Carlton,15598
1,Docklands,15680
3,Melbourne (CBD),83853
